In [ ]:
import gymnasium as gym
import random

In [ ]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")
env = TransformReward(env, lambda r: 0.01*r)
observation, info = env.reset()
#step result:
#(square, reward, end, idk, idk)

In [ ]:
from gymnasium.envs.toy_text.frozen_lake import FrozenLakeEnv

class ModifiedFrozenLakeEnv(FrozenLakeEnv):
    def __init__(self, **kwargs):
        super(ModifiedFrozenLakeEnv, self).__init__(**kwargs)
        # Change the rewards according to your requirement
        self.rewards = {'S': -1, 'F': 0, 'H': -10, 'G': 10}
        self.reward_range = (-10, 10)

    def step(self, action):
        state, reward, done, info = super().step(action)
        # Modify the reward based on the current state
        reward = self.rewards[self.desc[state]]
        return state, reward, done, info

env = ModifiedFrozenLakeEnv()

In [ ]:
#panic switch
env.close()

In [ ]:
def create_Q_table():
    q_table = {}
    # Populate Q-table with zeros for each state-action pair
    for i in range(20):
        q_table[i] = {0: 0, 1: 0, 2: 0, 3: 0}
    return q_table
        
#L = 0, D = 1, R = 2, U = 3

In [ ]:
observation, info = env.reset()

In [ ]:
#init vars
alpha = 0.5 #α
gamma = 0.5 #γ
epsilon = 0.3 #ε
episodeMax = 1000
num_columns = 4
num_rows = 5

In [ ]:
grid = [[1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12],
        [13, 14, 15, 16],
        [17, 18, 19, 20]]

In [ ]:
q_table[square][direction] = q_table[square][direction] + alpha * (reward + gamma * max(q_table[newSquare]) - q_table[square][direction])

In [ ]:
(q_table[square][0] == q_table[square][1]) and (q_table[square][2] == q_table[square][3])

In [ ]:
def find_row_col(grid, value):
    for row_index, row in enumerate(grid):
        if value in row:
            col_index = row.index(value)
            return row_index, col_index

In [ ]:
q_table = create_Q_table()
for i in range(episodeMax):#looping through episodes
    terminal = False
    observation, info = env.reset()
    square = 0
    while(not terminal):
        allSame = (q_table[square][0] == q_table[square][1]) and (q_table[square][2] == q_table[square][3])
        if(random.randint(0, 10) > epsilon * 10 and not allSame): #check if all values are same 
            direction = max(q_table[square])
        else:
            direction = random.randint(0, 3)
        row, col = find_row_col(grid, square+1)  
        if(direction == 0 and col == 0):
            continue
        elif(direction == 1 and row == 4):
            continue
        elif(direction == 2 and col == 3):
            continue
        elif(direction == 3 and row == 0):
            continue
        arr = env.step(direction)
        newSquare = arr[0]
        reward = arr[1]
        q_table[square][direction] = q_table[square][direction] + alpha * (reward + gamma * q_table[newSquare][max(q_table[newSquare])] - q_table[square][direction])
        terminal = arr[2]
        square = newSquare
        if(reward == 1):
            print("got reward!")
q_table